In [19]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [ ]:
############################################ government canada hourly historical weather #######################################
############### Downloads hourly weather data from government canada website, for user specified time period ####################

# Example Download url: http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=48549&Year=2018&Month=9&Day=13&timeframe=1&submit=Download+Data
# Example Page url: http://climate.weather.gc.ca/climate_data/hourly_data_e.html?StationID=48549
# Glossary: http://climate.weather.gc.ca/glossary_e.html

#48549 - Toronto City Centre - missing data?
#31688 - Toronto City


In [20]:
engine = create_engine("postgresql://localhost/weather")
df.to_sql('historicals', engine)

ModuleNotFoundError: No module named 'psycopg2'

In [49]:
df = download_data(2018,2018,1,12)

In [50]:
df.head()

,year,month,day,time,drybulb,dewpoint,relative_humidity,wind_dir_10s_deg,wind_spd_kmh,visibility_km,stn_press_kpa,hmdx,wind_chill,weather
datetime,,,,,,,,,,,,,,
2018-01-01 00:00,2018,1,1,NaN,-16.9,-21.1,70.0,NaN,NaN,NaN,101.64,NaN,NaN,NaN
2018-01-01 01:00,2018,1,1,NaN,-16.3,-20.6,69.0,NaN,NaN,NaN,101.61,NaN,NaN,NaN
2018-01-01 02:00,2018,1,1,NaN,-17.6,-21.5,71.0,NaN,NaN,NaN,101.67,NaN,NaN,NaN
2018-01-01 03:00,2018,1,1,NaN,-18.6,-22.0,75.0,NaN,NaN,NaN,101.63,NaN,NaN,NaN
2018-01-01 04:00,2018,1,1,NaN,-17.8,-21.1,75.0,NaN,NaN,NaN,101.60,NaN,NaN,NaN


In [48]:
def download_data(startyr,endyr,startmt,endmt):
   
    df = pd.DataFrame()
    years = list(range(startyr,endyr+1))
    
    for year in years:
        # Starts at startmt for the first year in years, likewise ends at endmt for last year in years
        startmt2 = startmt if year == startyr else 1
        endmt2 = endmt if year == endyr else 12
       
        year_df = pd.DataFrame()
        months = list(range(startmt2,endmt2+1))
        yr = year
        
        for month in months: 
            
            mo = month       
            header = 14 if month >= 4 and year >= 2018 else 13
            mo = month
            station_id = 31688 # stations can be found at http://climate.weather.gc.ca/historical_data/search_historic_data_e.html 

            url = 'http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID='+str(station_id)+'&Year='+str(yr)+'&Month='+str(mo)+'&Day=30&timeframe=1&submit=Download+Data'
            month_df = pd.read_csv(url, header = header)

            # drops 'flag' columns:
            month_df.drop(
                            ["Year":"year",
                            "Month":"month",
                            "Day":"day",
                            "Time":"time",
                            'Temp Flag', 
                            'Visibility Flag',
                            'Stn Press Flag',
                            'Hmdx Flag',
                            'Wind Chill Flag', 
                            'Dew Point Temp Flag', 
                            'Rel Hum Flag',
                            'Wind Dir Flag',
                            'Wind Spd Flag'], 
                          axis =1, 
                          inplace = True
                         )
            # renames columns:
            month_df = month_df.rename(index=str, 
                                       columns={

                                           "Date/Time":"datetime",
                                           "Temp (°C)": "drybulb",
                                           "Dew Point Temp (°C)":"dewpoint",
                                           "Rel Hum (%)" : "relative_humidity",
                                           "Wind Dir (10s deg)" : "wind_dir_10s_deg",
                                           "Wind Spd (km/h)" : "wind_spd_kmh",
                                           "Visibility (km)" : "visibility_km",
                                           "Stn Press (kPa)" : "stn_press_kpa",
                                           "Hmdx" : "hmdx",
                                           "Wind Chill" : "wind_chill",
                                           "Weather" : "weather"
                                        
            })

            # Sets index to DateTime:
            month_df = month_df.set_index('datetime')
            
            # Changes hour to an integer
            month_df['time'] = pd.to_numeric(month_df['time'].fillna(0), errors='coerce')

            # Appends month to year
            year_df = year_df.append(month_df)

        today = str(datetime.date.today())
        year_df = year_df.loc[(year_df.index < today)]
    
        # appends year to total
        df = df.append(year_df)
        
        # adds Weekday column (used for load forecaster model)
        # weekday = "DateTime".weekday()
        # df.insert(loc = 3, column = 'Weekday', value = 0)
        
    # Saving to .xlsx converts datetimes to strings. Not recommended.
    ### year_df.to_excel(r"C:\Users\cheide\Desktop\weather_station_"+str(station_id)+".xlsx", encoding='utf-8')
    
    df.to_csv("~//Desktop//station"+str(station_id)+"_"+str(startyr)+"-"+str(startmt)+" to "+str(endyr)+"-"+str(endmt)+".csv", encoding='utf-8')
    return df
    

In [41]:
# To locate a specfic cell :
df.loc['2017-11-01 00:00','DB Temp']
# Can't seem to select index this way. Might want to create a separate index so that 
#   the DateTime field can be manipulated (e.g. to get Weekday).
# The line below brings up 'index' error:
#   df2 = df.apply(lambda row : row['Index'].weekday(), axis = 1)

5.4000000000000004